In [1]:
import os
import numpy as np
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

consumption_dir = r'C:\Thesis\data\data_full\per_day\data_Consumption_per_day.pkl'
injection_dir = r'C:\Thesis\data\data_full\per_day\data_Injection_per_day.pkl'
offtake_dir = r'C:\Thesis\data\data_full\per_day\data_Offtake_per_day.pkl'
weather_dir =  r'C:\Thesis\data\data_full\weather_data2016-2021.pkl'
master_dir =  r'C:\Thesis\data\data_full\master_full_preprocessed.pkl'

consumption = pd.read_pickle(consumption_dir)
injection = pd.read_pickle(injection_dir)
offtake = pd.read_pickle(offtake_dir)
weather = pd.read_pickle(weather_dir)
master = pd.read_pickle(master_dir)

In [5]:
def data_df_to_quantiles(data_df): 
    # q = np.concatenate([np.arange(0, 0.05, 0.01), np.arange(0.05, 0.96, 0.05), np.arange(0.95, 1.005, 0.01)])
    q = np.arange(0, 1.01, 0.05)
    quantiles = (
        data_df.quantile(q, interpolation = 'nearest')
        .set_axis((q*100).astype('int'), axis = 0)
    )
    lower_quantiles = quantiles.loc[:49, :].stack().to_frame('min').rename_axis(index = ['lower_quantile', 'timestamp'])
    upper_quantiles = quantiles.loc[51:, :].stack().to_frame('max').reset_index(level = 0).rename(columns = {'level_0':'upper_quantile'}).assign(lower_quantile = lambda x: 100-x.upper_quantile).set_index('lower_quantile', append = True).swaplevel(0,1).sort_index().rename_axis(index = ['lower_quantile', 'timestamp'])
    return lower_quantiles.join(upper_quantiles).reset_index().assign(quantiles = lambda x: x.lower_quantile.astype('str').str.zfill(2)+'-'+x.upper_quantile.astype('str').str.zfill(2)).drop(columns = ['lower_quantile', 'upper_quantile'])

In [20]:
injection_temp = injection.reset_index().drop(columns=['date'])
injection_temp['ID']=1
plot_df_injection = injection_temp.groupby('ID').apply(data_df_to_quantiles).droplevel(1).reset_index()[1:]
plot_df_injection['timestamp'] = plot_df_injection['timestamp'].apply(lambda x: str(x))
area_chart = alt.Chart(plot_df_injection).mark_area().encode(
    x = 'timestamp', 
    y= alt.Y('min:Q',title = 'consumption (kWh)',axis = alt.Axis(format = '.2f')),  
    y2='max:Q', 
    color= 'quantiles:O',    
    column = alt.Column('ID',  header=alt.Header(labelFontSize=15))
).resolve_scale(y = 'shared').resolve_axis(y= 'independent').interactive(bind_x = False).configure_legend(labelLimit= 0)
area_chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [17]:
offtake_temp = offtake.reset_index().drop(columns=['date'])
offtake_temp['ID']=1
plot_df_offtake = offtake_temp.groupby('ID').apply(data_df_to_quantiles).droplevel(1).reset_index()[1:]
plot_df_offtake['timestamp'] = plot_df_offtake['timestamp'].apply(lambda x: str(x))
area_chart = alt.Chart(plot_df_offtake).mark_area().encode(
    x = 'timestamp', 
    y= alt.Y('min:Q',title = 'consumption (kWh)',axis = alt.Axis(format = '.2f')),  
    y2='max:Q', 
    color= 'quantiles:O',    
    column = alt.Column('ID',  header=alt.Header(labelFontSize=15))
).resolve_scale(y = 'shared').resolve_axis(y= 'independent').interactive(bind_x = False).configure_legend(labelLimit= 0)
area_chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [18]:
consumption_temp = consumption.reset_index().drop(columns=['date'])
consumption_temp['ID']=1
plot_df_consumption = injection_temp.groupby('ID').apply(data_df_to_quantiles).droplevel(1).reset_index()[1:]
plot_df_consumption['timestamp'] = plot_df_consumption['timestamp'].apply(lambda x: str(x))
area_chart = alt.Chart(plot_df_consumption).mark_area().encode(
    x = 'timestamp', 
    y= alt.Y('min:Q',title = 'consumption (kWh)',axis = alt.Axis(format = '.2f')),  
    y2='max:Q', 
    color= 'quantiles:O',    
    column = alt.Column('ID',  header=alt.Header(labelFontSize=15))
).resolve_scale(y = 'shared').resolve_axis(y= 'independent').interactive(bind_x = False).configure_legend(labelLimit= 0)
area_chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [2]:
full_year = r'C:\Thesis\data\data_full\data_full_filtered.pkl'
full_year = pd.read_pickle(full_year).reset_index()
full_year['Datetime'] = full_year['Datetime'].apply(lambda x: x.strftime('%H:%M:%S'))
full_year = full_year.pivot_table(index = ['ID'], columns = ['Datetime'], values = 'Consumption')

In [3]:
full_year = full_year.reset_index()
full_year['ID'] = 1

In [6]:
plot_df = full_year.groupby('ID').apply(data_df_to_quantiles).droplevel(1).reset_index()
# plot_df['timestamp'] = plot_df['timestamp'].apply(lambda x: x.strftime('%H:%M:%S'))

In [19]:
area_chart = alt.Chart(plot_df).mark_area().encode(
        x = alt.X('timestamp',title='time', axis=alt.Axis(labels=False)), 
        y= alt.Y('min:Q',title = 'consumption (kWh)',axis = alt.Axis(format = '.2f')),  
        y2='max:Q', 
        color= 'quantiles:O'
    ).properties(width=700,
                 height=500
    ).resolve_scale(y = 'shared').resolve_axis(y= 'independent').interactive(bind_x = False).configure_legend(labelLimit= 0)

In [20]:
area_chart

alt.Chart(...)